# Notebook #5

Jeffrey Sutherland
8/31/2021

In [1]:
import pandas as pd
import numpy as np
import pickle
import numpy
import os
from collections import defaultdict
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import l1_min_c
from sklearn.model_selection import train_test_split

In [2]:
PATH_DATA_INPUT = "../data/input/"
PATH_DATA_INTERMEDIATE = "../data/intermediate/"
PATH_DATA_OUTPUT = "../data/output/"
PATH_MODELS = '../data/models/'

Set the number of trials (folds) of leave 20% out cross validation
testmode = True causes data frames to be written out

In [3]:
numtrials = 50
testmode = False

read the adverse event (FAERS, SIDER) positive and negatives from the pickled output of make_AE_training_sets.ipynb

In [4]:
with open(PATH_DATA_INTERMEDIATE + 'AE_training_sets.pkl', 'rb') as inf:
    AE_data = pickle.load(inf)

print('Read in AE pickle; top level keys are {}'.format(AE_data.keys()))

Read in AE pickle; top level keys are dict_keys(['SIDER:PT', 'SIDER:HT', 'SIDER:HG', 'FAERS:PT', 'FAERS:HT', 'FAERS:HG', 'SIDER_DRUGS', 'FAERS_DRUGS'])


read the picked activity dataset from make_AE_vs_activity_dataset.ipynb

In [5]:
with open(PATH_DATA_INTERMEDIATE + 'activity_training_sets_pub.pkl', 'rb') as inf:
    assay_data = pickle.load(inf)

print('Read in assay pickle; number of assay groups at top level: {}'.format(len(assay_data)))

Read in assay pickle; number of assay groups at top level: 168


read the thresholds to apply to IC50 and margins - established by analyzing data from the calc_AE_vs_assay_score.ipynb
notebook

In [6]:
# input from the univartiate analysis done in spotfire
thresholds = pd.read_csv(PATH_DATA_INPUT + 'assay_vs_param_thresholds_update.txt', sep='\t')
thresholds.head(5)

,assay_group,param,threshold
0,10942,cmax_margin,10
1,10942,ic50,30
2,10943,ic50,30
3,13013,cmax_margin,10
4,13013,free_cmax_margin,100


read the dataset of AE MedDRA codes vs. assay groups to use in the model building process

In [7]:
ae_assay_pairs = pd.read_csv(PATH_DATA_INPUT + 'AE_vs_assay_pairs_logistic_modelling_update.txt', sep='\t')
ae_assay_pairs.head(5)

,code,type,assay,source,significant types
0,10000125,PT,3124,SIDER,ic50
1,10000125,PT,"3197, 41523, 42786",SIDER,"cmax_margin, free_cmax_margin, ic50"
2,10000125,PT,5426,SIDER,ic50
3,10000389,PT,3124,SIDER,ic50
4,10000389,PT,3126,SIDER,free_cmax_margin


Function to prepare a dataframe with AE coded as True/False and columns for the selected parameters and assay combinations

When performing statistical associations between AEs and activity results, results with IC50 qualifier (and hence
Cmax and free Cmax margin) are censored according to the tresholds read in from prior computation

Only assay + activity result types having 70% or more of the maximum assay result count (for margin-type results) is
retained

Returns a dataframe

In [8]:
def make_model_df(code, tp, source, assay_vs_type):

    pair = source + ':' + tp
    if AE_data.get(pair, None) is None:
        print('Failed to find AE info for pair {}; skipping'.format(pair))
        return None

    pos = list(AE_data[pair][code]['pos'])
    neg = list(AE_data[pair][code]['neg'])
    both = pos + neg
    outcomes = [1 for x in pos] + [0 for y in neg]
    df = pd.DataFrame(zip(both, outcomes), columns = ['struct_id', 'outcomes'])
    # track compounds that have at least one margin value; don't want a dataframe with NaNs for all drugs where
    # cmax, PPB was not available (i.e. we require at least one margin - free or total - to be significant in this analysis)
    have_margin = set()

    # track number of margin values by assay, to remove assays with too many missing values
    assay_count = dict()

    for assay in assay_vs_type['assay'].unique():

        # trying to use all 3 result types regardless of whether they were individually significant at our univariate cutoff
        # because the tresholds data frame excludes any assay - parameter type without any significant associations
        # at p-kw <= 1E-03, there will be logging below indicating skipped pairs
        sig_types = ['ic50', 'cmax_margin', 'free_cmax_margin']

        for param in sig_types:
            # get the pre-selected cutoff for this assay and result type (ic50, cmax_margin, free_cmax_margin) pair
            try:
                cutoff = thresholds[ (thresholds['assay_group'] == assay) & (thresholds['param'] == param) ]['threshold'].iloc[0]
            except:
                print('Failed to get cutoff for assay {} vs. param type {}; skipping'.format(assay, param))
                continue

            label = assay + '|' + param
            simplified = dict()
            numvals = 0
            for c in both:
                if assay_data[assay].get(c, None) is None:
                    # no data
                    continue
                elif assay_data[assay][c][param] is None or numpy.isnan(assay_data[assay][c][param]):
                    # happens for Cmax or free Cmax where IC50 was avail but not the exposure, hence these are NaN
                    continue
                elif assay_data[assay][c]['prefix'] == '>' and assay_data[assay][c][param] < cutoff:
                    continue
                elif assay_data[assay][c][param] > cutoff:
                    simplified[c] = np.log10(cutoff)
                    numvals += 1
                    if param in ['free_cmax_margin', 'cmax_margin']:
                        have_margin.add(c)
                else:
                    simplified[c] = np.log10(assay_data[assay][c][param])
                    numvals += 1
                    if param in ['free_cmax_margin', 'cmax_margin']:
                        have_margin.add(c)

            assay_count[label] = numvals
            df[label] = df['struct_id'].map(simplified)

    # because all 3 activity measures are used for each assay, this is an OK way to find the subset
    # of compounds that have margins.  With the original as intended approach (i.e. receiving the
    # measure as a parameter), a single margin-based measure could be too contraining and result in the
    # eliminatinion of many drugs - i.e. an assay with little data for which compounds with assay data
    # and human Cmax << compounds with human Cmax
    df = df[ df['struct_id'].isin(have_margin) ]

    if testmode:
        filename = PATH_MODELS + 'model_matrix_preimpute_code_{}_source_{}.csv'.format(code, source)
        df.to_csv(filename, index=False)

    # determine which assay (for margin-type results) has the max result count
    maxmargin = next(filter(lambda x: 'margin' in x, sorted(assay_count.keys(), key=lambda x: assay_count[x], reverse=True)))
    maxcount = assay_count[maxmargin]

    dropcols = list()
    for assay in assay_count:
        if assay_count[assay]/maxcount < 0.7:
            dropcols.append(assay)
        # use median assay value where missing
        else:
            df[assay]=df[assay].fillna(df[assay].median())

    print('Removing columns with more than 30% missing values: {}'.format(dropcols))
    df.drop(columns=dropcols, inplace=True)

    return df

Function to build lasso (l1)-penalized logistic regression models from data frame using dataframe with struct_id
on column 1, AE category on column 2 and assays on 3+

Returns a dictionary with various results.  See paper methods

adapted from https://chrisalbon.com/code/machine_learning/logistic_regression/logistic_regression_with_l1_regularization/
and https://scikit-learn.org/stable/auto_examples/linear_model/plot_logistic_path.html


In [9]:
def build_model_from_df(model_df):

    y = model_df.iloc[:,1]
    X = model_df.iloc[:,2:]

    # sequence of penalization parameters 'c' for l1 penalized logistic regression
    c_seq = l1_min_c(X, y, loss='log')*np.logspace(0, 5, 20)
    clf = LogisticRegression(penalty='l1', solver='saga', max_iter=10000, warm_start=True)

    test_accuracy = defaultdict(list)
    nonzero_coefs = defaultdict(dict)
    nonzero_coef_count = defaultdict(list)
    features = list(X.columns)

    trial = 0

    while trial < numtrials:

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # skip any splits without both classes in train and test - ROC AUC calc fails
        if len(np.unique(y_test)) < 2 or len(np.unique(y_train)) < 2:
            continue

        for c in c_seq:
            clf.set_params(C=c)
            clf.fit(X_train, y_train)
            these_coefs = clf.coef_.ravel().copy()
            nonzero_coef_count[c].append(np.count_nonzero(these_coefs))
            roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
            test_accuracy[c].append(roc_auc)

            # track which assays have non-zero coefs at each iteration
            for i in range(len(these_coefs)):
                if these_coefs[i] != 0:
                    feature = features[i]
                    nonzero_coefs[c][feature] = nonzero_coefs[c].get(feature, 0) + 1
        trial += 1

    summary_stats = list()
    max_error_adjuted_accuracy = 0
    max_accuracy_c = None
    sparse_accuracy_c = None

    # summarize results at each L1 penalty
    for c in c_seq:
        # get average and stderr accuracy
        avg_accuracy = np.mean(test_accuracy[c])
        sem_accuracy = np.std(test_accuracy[c], ddof=1)/np.sqrt(numtrials)
        avg_feature_count = np.mean(nonzero_coef_count[c])
        error_adjusted_accuracy =  avg_accuracy - sem_accuracy

        if error_adjusted_accuracy > max_error_adjuted_accuracy:
            max_error_adjuted_accuracy = error_adjusted_accuracy
            max_accuracy_c = c

        summary_stats.append({'c': c, 'is_max_AUC': False, 'is_1se_AUC': False, 'avg_AUC': avg_accuracy,
                              'sem_AUC': sem_accuracy, 'avg_feature_count': avg_feature_count, 'n': numtrials})

    # loop back through c values and annotate them as the best one or the one within 1se of best
    for r in summary_stats:
        if sparse_accuracy_c is None and r['avg_AUC'] >= max_error_adjuted_accuracy:
            r['is_1se_AUC'] = True
            sparse_accuracy_c = r['c']
        if r['c'] == max_accuracy_c:
            r['is_max_AUC'] = True

    if sparse_accuracy_c is None:
        print('Failed to find c giving model within 1SE of best model; error')
        return None

    # build final models on complete set to determine order of variable (feature) appearance into model
    variable_order = dict()
    variable_count = 0
    sparse_coefs = dict()
    max_coefs = dict()

    for c in c_seq:
        clf.set_params(C=c)
        clf.fit(X, y)
        these_coefs = clf.coef_.ravel()

        # track which assays have non-zero coefs at each iteration; multiple can appear on same iteration
        new_variables = list()
        for i in range(len(these_coefs)):

            feature = features[i]
            if c == sparse_accuracy_c:
                sparse_coefs[feature] = these_coefs[i]

            if c == max_accuracy_c:
                max_coefs[feature] = these_coefs[i]

            if these_coefs[i] != 0 and variable_order.get(feature, None) is None:
                new_variables.append(feature)

        if not new_variables:
            continue

        # if variables are tied by order of appearance (come in at same penalty c value), use avg rank
        avg_rank = np.mean(range(variable_count+1, len(new_variables)+variable_count+1))
        variable_count += len(new_variables)
        for v in new_variables:
            variable_order[v] = avg_rank

    results = {'CV_stats': summary_stats, 'CV_variable_use': nonzero_coefs, 'max_AUC_c': max_accuracy_c,
               '1se_AUC_c': sparse_accuracy_c, 'variables':[]}
    for v in features:
        # it's possible to have some dataset variables that fail to be used at any shrinkage param - use defaults below
        results['variables'].append({'variable': v, 'variable_order': variable_order.get(v, 999),
                                     '1se_AUC_coef': sparse_coefs.get(v, 0), 'max_AUC_coef': max_coefs.get(v, 0)})
    return results

In [10]:
combined_cv_stats = list()
combined_variables = list()
combined_cv_variables = list()

for code in ae_assay_pairs['code'].unique():

    # code type is 1:1 with code; just take first one
    tp = ae_assay_pairs[ (ae_assay_pairs['code'] == code) ]['type'].iloc[0]
    for source in ae_assay_pairs[ (ae_assay_pairs['code'] == code)]['source'].unique():

        assay_vs_type = ae_assay_pairs[ (ae_assay_pairs['code'] == code) & (ae_assay_pairs['source'] == source) ][['assay', 'significant types']]
        model_df = make_model_df(code, tp, source, assay_vs_type)

        if testmode:
            filename = PATH_MODELS + 'model_matrix_code_{}_source_{}.csv'.format(code, source)
            model_df.to_csv(filename, index=False)

        model_results = build_model_from_df(model_df)

        if model_results is None:
            print('Error in building models for code {} vs. source {} pair; skipping'.format(code, source))
            continue

        for r in model_results['CV_stats']:
            r['code'] = code
            r['source'] = source
            combined_cv_stats.append(r)

        for r in model_results['variables']:
            r['code'] = code
            r['source'] = source
            combined_variables.append(r)

        c_count = 0
        for c in model_results['CV_variable_use']:
            c_count += 1
            for v in model_results['CV_variable_use'][c]:
                combined_cv_variables.append({'c': c, 'c_seq': c_count, 'variable': v, 'count': model_results['CV_variable_use'][c][v], 'code': code, 'source': source})

        print('Done code {} vs. source {}'.format(code, source))

Failed to get cutoff for assay 5426 vs. param type free_cmax_margin; skipping
Removing columns with more than 30% missing values: []
Done code 10000125 vs. source SIDER
Failed to get cutoff for assay 3132 vs. param type cmax_margin; skipping
Failed to get cutoff for assay 3132 vs. param type free_cmax_margin; skipping
Failed to get cutoff for assay 3160 vs. param type cmax_margin; skipping
Failed to get cutoff for assay 3160 vs. param type free_cmax_margin; skipping
Removing columns with more than 30% missing values: ['3124|cmax_margin', '3124|free_cmax_margin', '3130|free_cmax_margin', '3131|free_cmax_margin', '3132|ic50', '3134, 18942|free_cmax_margin', '3157|free_cmax_margin', '3160|ic50', '3483, 41521, 42787|cmax_margin', '3483, 41521, 42787|free_cmax_margin', '41704, 42863|cmax_margin', '41704, 42863|free_cmax_margin', '5363|ic50', '5363|cmax_margin', '5363|free_cmax_margin']
Done code 10000389 vs. source SIDER
Removing columns with more than 30% missing values: ['3130|free_cmax_m

From this output, the 1se_coeff is what we analyze later

In [11]:
df1 = pd.DataFrame(combined_cv_stats)
df1.to_csv(PATH_DATA_OUTPUT + 'multivariate_cv_stats_update.csv', index=False)

df2 = pd.DataFrame(combined_variables)
df2.to_csv(PATH_DATA_OUTPUT + 'multivariate_final_variable_update.csv', index=False)

df3 = pd.DataFrame(combined_cv_variables)
df3.to_csv(PATH_DATA_OUTPUT + 'multivariate_cv_variable_update.csv', index=False)

In [12]:
print('Script complete')

Script complete
